# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/qTEA1/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298265078243                   -0.85    5.0         
  2   -8.300151215237       -2.72       -1.25    1.0    138ms
  3   -8.300424111512       -3.56       -1.89    2.2    118ms
  4   -8.300461951489       -4.42       -2.75    3.0    132ms
  5   -8.300464308928       -5.63       -3.13    3.1    148ms
  6   -8.300464481570       -6.76       -3.30    1.5    109ms
  7   -8.300464567001       -7.07       -3.45    1.4    117ms
  8   -8.300464612958       -7.34       -3.60    1.2    128ms
  9   -8.300464636005       -7.64       -3.79    1.8    109ms
 10   -8.300464641628       -8.25       -3.96    1.8    109ms
 11   -8.300464643835       -8.66       -4.29    1.6    109ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67421013471                   -0.70    6.0         
  2   -16.67860884724       -2.36       -1.14    2.2    287ms
  3   -16.67920246114       -3.23       -1.86    2.9    324ms
  4   -16.67927617370       -4.13       -2.74    2.4    287ms
  5   -16.67928582890       -5.02       -3.18    5.8    425ms
  6   -16.67928618283       -6.45       -3.45    2.4    306ms
  7   -16.67928621432       -7.50       -3.91    1.9    257ms
  8   -16.67928622130       -8.16       -4.57    2.8    314ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32496366922                   -0.56    6.0         
  2   -33.33261794687       -2.12       -1.00    1.0    925ms
  3   -33.33396176980       -2.87       -1.74    4.4    1.26s
  4   -33.33424210342       -3.55       -2.63    2.1    1.03s
  5   -33.33569029718       -2.84       -2.28    5.9    1.53s
  6   -33.33694210613       -2.90       -2.52    7.0    1.31s
  7   -33.33694201224   +   -7.03       -2.54    1.0    897ms
  8   -33.33687237965   +   -4.16       -2.41    1.2    920ms
  9   -33.33685067169   +   -4.66       -2.35    1.1    898ms
 10   -33.33684953923   +   -5.95       -2.35    1.1    903ms
 11   -33.33686303512       -4.87       -2.39    1.5    915ms
 12   -33.33686626033       -5.49      

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298412369937                   -0.85    5.1         
  2   -8.300253972619       -2.73       -1.59    1.0   78.5ms
  3   -8.300434664414       -3.74       -2.61    3.4    116ms
  4   -8.300405094259   +   -4.53       -2.36    5.4    180ms
  5   -8.300464238257       -4.23       -3.43    1.0   85.0ms
  6   -8.300464568768       -6.48       -3.79    3.4    139ms
  7   -8.300464639238       -7.15       -4.30    1.4   94.2ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32562229069                   -0.56    7.4         
  2   -33.32152433774   +   -2.39       -1.27    1.0    771ms
  3   -19.76129331536   +    1.13       -0.50    6.0    1.60s
  4   -33.30952776285        1.13       -1.69    6.6    1.73s
  5   -33.22993262448   +   -1.10       -1.31    2.8    1.19s
  6   -33.05910212048   +   -0.77       -1.34    3.6    1.27s
  7   -33.28710846610       -0.64       -1.70    4.0    1.16s
  8   -33.33545504853       -1.32       -2.32    2.4    954ms
  9   -33.33631751711       -3.06       -2.38    3.1    1.25s
 10   -33.33669428446       -3.42       -2.77    2.0    986ms
 11   -33.33688105501       -3.73       -3.12    3.4    1.08s
 12   -33.33693757823       -4.25      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.